## 5. Transformer Finetuning
To create our state operator classifier, we will be using the HuggingFace Transformers library to finetune the DistilBERT model. DistilBERT is an optimized, distilled version of BERT, which is a large Transformer language model created by Google in 2018. Obviously, larger and more capable language models currently exist, but we want to use the minimum-size model that still succeeds on our task.

### 5.1 Setup

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModel
from transformers import AutoTokenizer

### 5.2 Prepare Datasets
We will use the HuggingFace custom Datasets library here. It has a range of features that make training easier.

#### 5.2.1 Load Data

In [2]:
from datasets import load_dataset

data_files = {"train":"../data/train.csv","test":"../data/test.csv"}
tweet_seqs = load_dataset("csv", data_files=data_files, lineterminator='\n')
tweet_seqs

Using custom data configuration default-c502a4b67625da65


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 2190.24it/s]


Dataset csv downloaded and prepared to /home/curttigges/.cache/huggingface/datasets/csv/default-c502a4b67625da65/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 1034.23it/s]


DatasetDict({
    train: Dataset({
        features: ['userid', 'tweet_text', 'tweet_time', 'clean_tweets', 'recent_tweets', 'operator', 'seq_id'],
        num_rows: 368567
    })
    test: Dataset({
        features: ['userid', 'tweet_text', 'tweet_time', 'clean_tweets', 'recent_tweets', 'operator', 'seq_id'],
        num_rows: 40951
    })
})

Let's take a look at an example of the data.

In [3]:
seq_sample = tweet_seqs["train"].shuffle(seed=42).select(range(1000))
seq_sample[:3]

{'userid': ['ANS_DevNeuro',
  'uqyBajgI70Y4eIp7bV2CLijqtEqc5+kivHt7MKmJo6I=',
  'Draganov313'],
 'tweet_text': ['Do not miss our ADNF Symposium VII \nThursday, April 29, 2021, 09:00 AM Brisbane \nhttps://t.co/42QoKXaKOb \nwith A/Pr… https://t.co/jwoVzTZgn0',
  'Venezuela is a leading light for eco-socialism, the future! Thank you @fundamiarbol! Socialism means a harmonious relationship between humans and nature, and Venezuela and Cuba are paving that path! https://t.co/enfnVIZeV5',
  'At the end of the meeting with Gen. Michael Kurilla, the new Commander-in-Chief of the US Central Command (Centcom)… https://t.co/DO0qsGStA3'],
 'tweet_time': ['2021-04-19 00:51:32+00:00',
  '2018-06-05 05:30',
  '2022-05-19 02:08:23+00:00'],
 'clean_tweets': ['Do not miss our ADNF Symposium VII Thursday April AM Brisbane with APr',
  'Venezuela is a leading light for ecosocialism the future Thank you Socialism means a harmonious relationship between humans and nature and Venezuela and Cuba are paving tha

From this, we can see that there are probably a number of shorter tweet sequences in the dataset--i.e., with fewer than 10 tweets. For consistency and since most real-world users have more than 10 tweets in their feed, we will exclude these sequences after marking them with a custom function.

#### 5.2.2 Remove Shorter Sequences

In [4]:
def compute_tweet_seq_len(example):
    """Compute number of tweets in tweet sequence
    
    Args:
        example (Dataset item): A single row item of the dataset

    Returns:
        dictionary: maps tweet sequence to count of tweets

    """
    return {"tweet_count": example["recent_tweets"].count("|")}

In [5]:
# compute number of tweets across entire dataset
tweet_seqs = tweet_seqs.map(compute_tweet_seq_len)
tweet_seqs["train"][0]

100%|██████████| 40951/40951 [00:01<00:00, 21479.12ex/s]


{'userid': '+TISRMUH0i0bKI15Nttok6AXOVgfei11wHRkbOP3R4=',
 'tweet_text': 'The best education in the world is that got by struggling to get a living.',
 'tweet_time': '2015-12-24 17:05',
 'clean_tweets': 'The best education in the world is that got by struggling to get a living',
 'recent_tweets': 'The best education in the world is that got by struggling to get a living | ',
 'operator': 1,
 'seq_id': 0,
 'tweet_count': 1}

In [6]:
# use the Datasets filter function to include only sequences with >9 tweets
tweet_seqs = tweet_seqs.filter(lambda x: x["tweet_count"] > 9)

100%|██████████| 41/41 [00:00<00:00, 144.90ba/s]


In [7]:
print(tweet_seqs.num_rows)

{'train': 282962, 'test': 31433}


#### 5.2.3 Train-Validation-Test Split
In addition to using the test-train split we made earlier, we will also set aside 20% of the training set for validation during the model training.

In [8]:
# map train, validation and test splits appropriately.
tweet_seqs_clean = tweet_seqs["train"].train_test_split(train_size=0.8, seed = 42)

tweet_seqs_clean["validation"] = tweet_seqs_clean.pop("test")

tweet_seqs_clean["test"] = tweet_seqs["test"]
tweet_seqs_clean

DatasetDict({
    train: Dataset({
        features: ['userid', 'tweet_text', 'tweet_time', 'clean_tweets', 'recent_tweets', 'operator', 'seq_id', 'tweet_count'],
        num_rows: 226369
    })
    validation: Dataset({
        features: ['userid', 'tweet_text', 'tweet_time', 'clean_tweets', 'recent_tweets', 'operator', 'seq_id', 'tweet_count'],
        num_rows: 56593
    })
    test: Dataset({
        features: ['userid', 'tweet_text', 'tweet_time', 'clean_tweets', 'recent_tweets', 'operator', 'seq_id', 'tweet_count'],
        num_rows: 31433
    })
})

In [11]:
# save the prepared dataset
tweet_seqs_clean.save_to_disk("../tweet-seqs")

Loading cached processed dataset at /home/curttigges/.cache/huggingface/datasets/csv/default-c502a4b67625da65/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-54253883bba44e37.arrow
Loading cached processed dataset at /home/curttigges/.cache/huggingface/datasets/csv/default-c502a4b67625da65/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-91eae106765d1788.arrow
Loading cached processed dataset at /home/curttigges/.cache/huggingface/datasets/csv/default-c502a4b67625da65/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-038f0b242740fc95.arrow


### 5.3 Load Model & Train
Next, we load the saved dataset, download the DistilBERT model, set training arguments, and finally finetune the model.

#### 5.3.1 Load Dataset From Disk

In [12]:
from datasets import load_from_disk

tweet_seqs_reloaded = load_from_disk("../tweet-seqs")
tweet_seqs_reloaded

DatasetDict({
    train: Dataset({
        features: ['userid', 'tweet_text', 'tweet_time', 'clean_tweets', 'recent_tweets', 'operator', 'seq_id', 'tweet_count'],
        num_rows: 226369
    })
    validation: Dataset({
        features: ['userid', 'tweet_text', 'tweet_time', 'clean_tweets', 'recent_tweets', 'operator', 'seq_id', 'tweet_count'],
        num_rows: 56593
    })
    test: Dataset({
        features: ['userid', 'tweet_text', 'tweet_time', 'clean_tweets', 'recent_tweets', 'operator', 'seq_id', 'tweet_count'],
        num_rows: 31433
    })
})

#### 5.3.2 Download Tokenizer & Tokenize sequences
HuggingFace language models generally come with their own tokenizers. We will need to use the tokenizer that comes with DistilBERT to tokenize our tweet sequences.

In [16]:
from transformers import DistilBertTokenizerFast, DataCollatorWithPadding

checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["recent_tweets"], truncation=True, padding=True)


tokenized_datasets = tweet_seqs_reloaded.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["userid", "tweet_text", "tweet_time", "clean_tweets","seq_id", "tweet_count"])
tokenized_datasets = tokenized_datasets.rename_column("operator", "labels")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

100%|██████████| 32/32 [00:03<00:00, 10.62ba/s]


Let's examine the results.

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['recent_tweets', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 226369
    })
    validation: Dataset({
        features: ['recent_tweets', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 56593
    })
    test: Dataset({
        features: ['recent_tweets', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 31433
    })
})

#### 5.3.3 Download Model & Selected Metrics

In [18]:
from transformers import TrainingArguments

training_args = TrainingArguments("../trainer-checkpoints", evaluation_strategy="epoch")

In [19]:
from transformers import DistilBertForSequenceClassification

# note the label count below--currently only 1/0 for state operator or not.
model = DistilBertForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [20]:
from datasets import load_metric

def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### 5.4 Set Arguments & Train

In [21]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Below is the final training command. Be cautious with the default commands below; it will save a checkpoint every 500 batches, and each checkpoint is almost 1 GB in size. It is therefore easy to accidentally fill up a smaller hard drive with the results.

In [17]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: recent_tweets. If recent_tweets are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/curttigges/miniconda3/envs/pytorch-dl-hf/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 226369
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 84891
  1%|          | 500/84891 [00:43<1:58:45, 11.84it/s]Saving model checkpoint to test-trainer/checkpoint-500
Configura

{'loss': 0.2988, 'learning_rate': 4.97055047060348e-05, 'epoch': 0.02}


Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
  1%|          | 1000/84891 [01:27<2:02:18, 11.43it/s]Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json


{'loss': 0.1709, 'learning_rate': 4.94110094120696e-05, 'epoch': 0.04}


Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
  2%|▏         | 1500/84891 [02:10<1:59:58, 11.58it/s]Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json


...

Model weights saved in test-trainer/checkpoint-84000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-84000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-84000/special_tokens_map.json
100%|█████████▉| 84500/84891 [2:07:27<00:31, 12.30it/s]Saving model checkpoint to test-trainer/checkpoint-84500
Configuration saved in test-trainer/checkpoint-84500/config.json


{'loss': 0.0017, 'learning_rate': 2.302953198807883e-07, 'epoch': 2.99}


Model weights saved in test-trainer/checkpoint-84500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-84500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-84500/special_tokens_map.json
100%|██████████| 84891/84891 [2:07:59<00:00, 13.84it/s]The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: recent_tweets. If recent_tweets are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 56593
  Batch size = 8
100%|██████████| 84891/84891 [2:10:58<00:00, 13.84it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 84891/84891 [2:10:58<00:00, 10.80it/s]

{'eval_loss': 0.003943989519029856, 'eval_accuracy': 0.9993992189846801, 'eval_f1': 0.9987627365356622, 'eval_runtime': 178.6127, 'eval_samples_per_second': 316.848, 'eval_steps_per_second': 39.611, 'epoch': 3.0}
{'train_runtime': 7858.099, 'train_samples_per_second': 86.421, 'train_steps_per_second': 10.803, 'train_loss': 0.023884876818861412, 'epoch': 3.0}


TrainOutput(global_step=84891, training_loss=0.023884876818861412, metrics={'train_runtime': 7858.099, 'train_samples_per_second': 86.421, 'train_steps_per_second': 10.803, 'train_loss': 0.023884876818861412, 'epoch': 3.0})

We see that the final model provides us with the following metrics, as evaluated on the validation dataset:
- **Accuracy:** 0.999
- **F1:** 0.999

#### Discussion

These values are unusually high, but have been found repeatedly with different sampling and data combinations. 

### 5.5 Saving Model

#### 5.5.1 Save Model Parameters

In [13]:
ft_model = "finetuned/troll_detect_distilbert"
trainer.save_model(ft_model)
tokenizer.save_pretrained(ft_model)

NameError: name 'trainer' is not defined

#### 5.5.2 Load Model For Inference
The following assumes you are using the uploaded model for inference and test set evaluation. The model is available on HuggingFace and Github.

In [53]:
#ft_model = "../finetuned/troll_detect_distilbert"
#model = AutoModel.from_pretrained(ft_model)
#tokenizer = AutoTokenizer.from_pretrained(ft_model)

In [55]:
from transformers import pipeline

classifier = pipeline(model="lingwave-admin/state-op-detector")

Downloading: 100%|██████████| 851/851 [00:00<00:00, 824kB/s]
https://huggingface.co/lingwave-admin/state-op-detector/resolve/main/config.json not found in cache or force_download set to True, downloading to /home/curttigges/.cache/huggingface/transformers/tmp1yimq2ox
Downloading: 100%|██████████| 671/671 [00:00<00:00, 836kB/s]
storing https://huggingface.co/lingwave-admin/state-op-detector/resolve/main/config.json in cache at /home/curttigges/.cache/huggingface/transformers/88c6143068bfc25bf3526a3020a6cda16cf69175c43f1603a6444ba46ba6895a.a03339a5f29705d5db21ac22e95d0c1f0fbdfc0f7d2e504a4e50ecec71ea25d5
creating metadata file for /home/curttigges/.cache/huggingface/transformers/88c6143068bfc25bf3526a3020a6cda16cf69175c43f1603a6444ba46ba6895a.a03339a5f29705d5db21ac22e95d0c1f0fbdfc0f7d2e504a4e50ecec71ea25d5
loading configuration file https://huggingface.co/lingwave-admin/state-op-detector/resolve/main/config.json from cache at /home/curttigges/.cache/huggingface/transformers/88c6143068bfc2

In [58]:
classifier("Washington Gives Ankara An Ultimatum | Trump Makes BigTime Overture To Iran | Authoritarian Spirits Congress The Espionage Act And Punishing WikiLeaks | Analysis Of The European Parliamentary Elections | One Mans Quest To Expose A Fake BBC Video About Syria | China Holds Three Trump Cards In War Against US | Maldives Affirms Fealty To Diego Garcia | The End Of Theresa May | Within The Church People Can Become Truly Free | China Hails Modi Victory This Is Why | ")

[{'label': 'State Operator', 'score': 0.9999926090240479}]

In [63]:
seq_sample = tweet_seqs["test"]['recent_tweets'][0:5]
seq_sample

['Seriously the account is private I can not see | Screenshot I can not see | Sorry boss e don finish | Still not close with all the makeup shes wearing Youre indirectly proving me right | Only if I lied | okay lets see if you can reply to this | I feel sad when you guys rate these celebs beauty over yours Deep down yk you can beat her | What I mean Bebe is exceptional | Shes not worth it right | Shes not that relevant | ',
 'VERSAILLES The British Empire had forty six fresh divisions in reserve | DISCLOSURE Mussolini was a secret agent of FONTAINEBLEAU the PowersthatBe in France | DISCLOSURE Ethiopia Haile Sellasie was a usurper a secret ally of Mussolini SECRET SOCIETIES | Mussolini in his time WW II tried to invade Ethiopia in memoriam of the eight thousand Italians were massacred slain in millions | Great War dubbed world war one Battle of Yser Ottoman Turks were annihilated by Prussia | Great War alias WW1 Prussia inflicted four millions casualties to the France of Pétain | HISTOR

In [69]:
results = classifier(tweet_seqs["test"]['recent_tweets'][0:100])

In [65]:
results

[{'label': 'Normal User', 'score': 0.9999964237213135},
 {'label': 'Normal User', 'score': 0.9999966621398926},
 {'label': 'Normal User', 'score': 0.9999970197677612},
 {'label': 'State Operator', 'score': 0.9999926090240479},
 {'label': 'State Operator', 'score': 0.9999926090240479}]

In [35]:
trainer.predict("one item")

***** Running Prediction *****
  Num examples = 8
  Batch size = 8


AttributeError: 'list' object has no attribute 'keys'

In [34]:
preds[2]

{'test_loss': 0.7067875862121582,
 'test_accuracy': 0.2506283205548309,
 'test_f1': 0.3673283016840805,
 'test_runtime': 103.8866,
 'test_samples_per_second': 302.57,
 'test_steps_per_second': 37.83}